# Dummy Net

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from utils.data import init_model_logging
import tensorflow as tf
import numpy as np
import pandas as pd
import os

## Animals Data

In [2]:
# [fur, extinction, weight] 
animals = np.array([[1.0, 0.7, 0.9], # lion
                    [0.8, 0.8, 1.0], # lion
                    [0.1, 0.3, 0.2], # seal
                    [0.0, 0.2, 0.1]  # seal
                   ], dtype=np.float32)

In [3]:
animal_labels = np.array(
                [[ 1.0],  # lion
                 [ 1.0],  # lion
                 [-1.0],  # seal
                 [-1.0]   # seal
                ], dtype=np.float32)

## Net Without Training 

### Build Graph

In [4]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
    
    w1 = tf.constant(np.array([[1, 2], [4, 5], [7, 8]], dtype=np.float32), name='w1_const')
    h = tf.matmul(x, w1)

    w2 = tf.constant(np.array([[1], [1]], dtype=np.float32), name='w2_const')
    y = tf.matmul(h, w2)

### Run Session

In [5]:
session = tf.Session(graph=graph)

In [6]:
prediction, = session.run([y], feed_dict={x: animals})
print('Prediction on animals:')
print(prediction)

Prediction on animals:
[[ 22.79999924]
 [ 24.60000038]
 [  6.        ]
 [  3.29999995]]


In [7]:
session.close()

## Net With Loss

### Build Graph

In [8]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
    label = tf.placeholder(tf.float32, shape=[None, 1], name='label')
    
    w1 = tf.constant(np.array([[1, 2], [4, 5], [7, 8]], dtype=np.float32), name='w1_const')
    h = tf.matmul(x, w1)

    w2 = tf.constant(np.array([[1], [1]], dtype=np.float32), name='w2_const')
    y = tf.matmul(h, w2)
    
    loss = tf.losses.mean_squared_error(label, y)

### Run Session

In [9]:
session = tf.Session(graph=graph)

In [10]:
loss_on_animal_data, = session.run([loss], feed_dict={x: animals, label: animal_labels})
print('Loss: {0:.2f}'.format(loss_on_animal_data))

Loss: 274.92


In [11]:
session.close()

## Net With Trainable Variables

### Build Graph

In [12]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
    label = tf.placeholder(tf.float32, shape=[None, 1], name='label')
    
    w1 = tf.get_variable('w1', initializer=tf.truncated_normal(shape=[3,2], stddev=0.1))
    h = tf.matmul(x, w1)

    w2 = tf.get_variable('w2', initializer=tf.truncated_normal(shape=[2,1], stddev=0.1))
    y = tf.matmul(h, w2)
    
    loss = tf.losses.mean_squared_error(label, y)
    initialize_vars = tf.global_variables_initializer()

### Run Session

In [13]:
session = tf.Session(graph=graph)

In [14]:
session.run(initialize_vars)
w1_values, w2_values = session.run([w1, w2], feed_dict={x: animals, label: animal_labels})
print('w1:')
print(w1_values)
print('w2:')
print(w2_values)

w1:
[[ 0.10959081 -0.13871621]
 [ 0.05815202 -0.01618201]
 [ 0.05937244 -0.11505231]]
w2:
[[ 0.01594922]
 [-0.03459523]]


In [ ]:
session.close()

## Net With Train Step

### Build Graph

In [ ]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
    label = tf.placeholder(tf.float32, shape=[None, 1], name='label')
    
    w1 = tf.get_variable('w1', initializer=tf.truncated_normal(shape=[3,2], stddev=0.1))
    h = tf.matmul(x, w1)

    w2 = tf.get_variable('w2', initializer=tf.truncated_normal(shape=[2,1], stddev=0.1))
    y = tf.matmul(h, w2)
    
    loss = tf.losses.mean_squared_error(label, y)
    train_step = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

    initialize_vars = tf.global_variables_initializer()

### Run Session

In [ ]:
session = tf.Session(graph=graph)
session.run(initialize_vars)

In [ ]:
for iteration in range(10001):
    _, loss_on_animal_data = session.run([train_step, loss], feed_dict={x: animals, label: animal_labels})
    if iteration % 500 == 0:
        print("Iteration {}: loss {}".format(iteration, loss_on_animal_data))

In [ ]:
prediction = session.run(y, feed_dict={x: animals})

In [ ]:
pd.DataFrame({'prediction':prediction[:,0], 'label':animal_labels[:,0]})

In [ ]:
session.close()